# Check SAFT-VR-Mie Implementations

In particular, check that outputs match calculated values with the SAFT-VR-Mie implementation in Clapeyron.jl

In [1]:
using Clapeyron
model = SAFTVRMie(["Propane"])
display(model.params.segment)
display(model.params.sigma)
display(model.params.epsilon)
display(model.params.lambda_r)
display(model.params.lambda_a)
display(Clapeyron.R̄)
# display(Clapeyron.Cλ(model, 0, 0, 0, 12, 6))
T = 400.0
rhomolar = 7000.0

SingleParam{Float64}("m") with 1 component:
 "Propane" => 1.6845

1×1 PairParam{Float64}(["Propane"]) with values:
 3.9056e-10

1×1 PairParam{Float64}(["Propane"]) with values:
 239.89

1×1 PairParam{Float64}(["Propane"]) with values:
 13.006

1×1 PairParam{Float64}(["Propane"]) with values:
 6.0

8.31446261815324

7000.0

In [2]:
# This is an optional setup cell for PyCall
# import Pkg; Pkg.add("PyCall")
using PyCall
ENV["PYTHON"] = ENV["CONDA_PREFIX"] * "/" * "/bin/python"
import Pkg; Pkg.build("PyCall")

    Building Conda ─→ `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/6e47d11ea2776bc5627421d59cdcc1296c058071/build.log`
    Building PyCall → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/53b8b07b721b77144a0fbbbc2675222ebf40a02d/build.log`


In [3]:
teqp = pyimport("teqp")
np = pyimport("numpy")
display(teqp.__version__)
j = Dict("kind" => "PCSAFT", "model" => Dict("names" => ["Propane"]))
model = teqp.make_model(j)
print( model.get_m(), model.get_sigma_Angstrom(), model.get_epsilon_over_k_K())
z = np.array([1.0])
Ar01 = model.get_Ar01(T, rhomolar, z)
R = 8.31446261815324
rhomolar*R*T*(1+Ar01), model.get_Ar00(T, rhomolar, z), model.get_splus(T, rhomolar*z)

"0.15.0dev"

[2.002][3.6184][208.11]

(9.187913506618721e6, -1.0291558679657231, 1.435330456977336)

In [4]:
j = Dict("kind" => "SAFT-VR-Mie", "model" => Dict("names" => ["Propane"]))
model = teqp.make_model(j)
display()
# print( model.get_m(), model.get_sigma_m(), model.get_epsilon_over_k_K())
z = np.array([1.0])
Ar01 = model.get_Ar01(T, rhomolar, z)
R = 8.31446261815324
display(model.solve_pure_critical(300, 1000))

display(model.get_core_calcs(T, rhomolar, z))
rhomolar*R*T*(1+Ar01), model.get_Ar00(T, rhomolar, z), model.get_splus(T, rhomolar*z)

(375.99449561839737, 4980.1962651166905)

Dict{Any, Any} with 13 entries:
  "alphar_mono"  => -0.882328
  "xs"           => [1.0]
  "a3kB3"        => -3.64976e5
  "mbar"         => 1.6845
  "zeta_x"       => 0.198169
  "dmat"         => [3.76333;;]
  "a2kB2"        => -8329.57
  "alphar_chain" => -0.111363
  "zeta_x_bar"   => 0.221504
  "rhoN"         => 4.2155e27
  "zeta"         => [0.00371808, 0.0139924, 0.0526579, 0.198169]
  "a1kB"         => -606.278
  "rhos"         => 0.00710101

(9.004986612957103e6, -0.9936903139741133, 1.4864584681864197)

In [5]:
import Pkg; Pkg.status("Clapeyron")
model = SAFTVRMie(["Propane"])
display(Clapeyron.crit_pure(model))
_d,_ρ_S,ζi,_ζ_X,_ζst,σ3x,m̄ = Clapeyron.data(model, 1/rhomolar, T, [1.0])
display(Dict("a_hs"=> Clapeyron.a_hs(model, 1/rhomolar, T, [1.0]),
"alphar_mono" => Clapeyron.a_mono(model, 1/rhomolar, T, [1.0]),
"alphar_chain" => Clapeyron.a_chain(model, 1/rhomolar, T, [1.0]),
"zeta" => ζi,
"zeta_x" => _ζ_X))
display(-Clapeyron.VT_entropy_res(model, 1/rhomolar, T, [1.0])/8.31446261815324)
B2_ = Clapeyron.second_virial_coefficient(model, T)
display(B2_)
display(_d)

Status `~/.julia/environments/v1.8/Project.toml`
  [7c7805af] Clapeyron v0.3.11 `https://github.com/ypaul21/Clapeyron.jl#master#master`


(375.9942376772184, 4.759562486477156e6, 0.00020079526183553885)

Dict{String, Any} with 5 entries:
  "zeta_x"       => 0.19817
  "a_hs"         => 1.76817
  "alphar_mono"  => -0.882324
  "alphar_chain" => -0.111364
  "zeta"         => (3.71808e27, 1.39924e18, 5.2658e8, 0.19817)

1.4864815367800055

-0.00018865097238816497

1-element Vector{Float64}:
 3.7633348428615264e-10

In [6]:
CP = pyimport("CoolProp.CoolProp")
-CP.PropsSI("Smolar_residual","T",T,"Dmolar",rhomolar,"Propane")/8.31446

1.5106425117743096

In [7]:
modelClap = SAFTVRMie(["Propane"])
j = Dict("kind" => "SAFT-VR-Mie", "model" => Dict("names" => ["Propane"]))
modelteqp = teqp.make_model(j)
quad = pyimport("scipy.integrate")

epskB = modelteqp.get_epsilon_over_k_K()[1] # [K]
sigma_m = modelteqp.get_sigma_m()[1] # [m]
lambda_r = modelteqp.get_lambda_r()[1]
lambda_a = modelteqp.get_lambda_a()[1]
C = lambda_r/(lambda_r-lambda_a)*(lambda_r/lambda_a)^(lambda_a/(lambda_r-lambda_a))

display("T d(teqp) d(Clapeyron) err%(teqp) err%(Clapeyron)")
for T in [50, 60, 70, 80, 90, 100, 120, 140, 160, 180, 195, 200, 205, 250, 300, 350, 400, 500, 600, 700, 800, 900, 1000]
    _d,_ρ_S,ζi,_ζ_X,_ζst,σ3x,m̄ = Clapeyron.data(modelClap, 1/rhomolar, T, [1.0])
    dClap = _d[:][1]*1e10
    dteqp = modelteqp.get_core_calcs(T, rhomolar, z)["dmat"][:][1]
    
    # The classical method based on adaptive quadrature
    function integrand(r_m)
        u = C*epskB*((sigma_m/r_m)^(lambda_r) - (sigma_m/r_m)^(lambda_a))
        return 1-exp(-u/T)
    end
    d_exact = quad.quad(integrand, 0, sigma_m, epsrel=1e-16, epsabs=1e-16)[1]*1e10

    ddiffteqp = abs(dteqp/d_exact-1.0)*100
    ddiffClap = abs(dClap/d_exact-1.0)*100
    
    display("$T $dteqp $dClap $(ddiffteqp)% $(ddiffClap)%")
end

"T d(teqp) d(Clapeyron) err%(teqp) err%(Clapeyron)"

"50 3.8773986983350106 3.877398699548621 4.473277304128942e-9% 2.682634114847815e-8%"

"60 3.872493920627443 3.8724939216594922 1.520885639649805e-8% 1.1441891878405386e-8%"

"70 3.867773310082607 3.8677733106918986 1.0726630694790629e-8% 5.026401517227441e-9%"

"80 3.8632197900666356 3.863219789785552 9.557665769932555e-9% 2.2817747691306067e-9%"

...

"600 3.722291690061605 3.722328134527149 1.2587890174664551e-6% 0.000977827970416456%"

"700 3.7048733818556094 3.704955349639191 4.741390557949643e-6% 0.00220768994640963%"

"800 3.6889837810844472 3.689140662858416 8.164394871279512e-6% 0.004244545035958502%"

"900 3.674358463250937 3.6746264209653416 1.0686250140334863e-5% 0.007281952409110559%"

"1000 3.66079856795589 3.6612193864309384 1.179074557988713e-5% 0.01148347231298974%"